## Library

In [1]:
import os
import zipfile
from utils.constants.main import header_companies, header_establishments, header_partners, path_raw_data

# Encontra o diretório do apache spark (utiliza a variável de ambiente $SPARK_HOME para pegar o diretório)
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, to_date, date_format, year
from pyspark.sql.types import DoubleType, StringType, DateType
import pyspark.sql.functions as func

## Functions

In [2]:
def createWorker():
    """
        Inicializa o servidor e configura o worker.
    """
    return SparkSession.builder \
        .master('local[*]') \
        .appName('Iniciando com Spark') \
        .config("spark.driver.bindAddress", "localhost") \
        .getOrCreate()

def downloadAndUnzipData(url, dest):
    os.system(f'wget {url} -P {dest} -q --show-progress')
    filename = url.split('/')[-1]
    zipfile.ZipFile(dest+'/'+filename, 'r').extractall(path=dest)

def sparkDataFrame(spark, files, header):
    df = spark.read.csv(files, sep=';', inferSchema=True)
    
    for key, column in enumerate(header):
        df = df.withColumnRenamed(f'_c{key}', column)

    return df

def totalRecords(df, name):
    print(f'Total {name}: {df.count()}')

def replaceValues(df, column, char, to_char):
    return df.withColumn(column, regexp_replace(column, char, to_char))

def convertType(df, column, to_type):
    return df.withColumn(column, col(column).cast(to_type))
    
def formatStringToDate(df, column, string_format='yyyyMMdd', to_format='dd/MM/yyyy'):
    return df.withColumn(column, date_format(to_date(col(column), string_format), to_format))
    

## Working with Spark Dataframe

### Create Spark Dataframe

In [3]:
spark = createWorker()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df_companies = sparkDataFrame(spark, '../raw-data/empresas/part-*', header_companies)
totalRecords(df_companies, 'Companies')

Total Companies: 4585679


In [5]:
df_establishments = sparkDataFrame(spark, '../raw-data/estabelecimentos/part-*', header_establishments)
totalRecords(df_establishments, 'Establishments')

Total Establishments: 4836219


In [6]:
df_partners = sparkDataFrame(spark, '../raw-data/socios/part-*', header_partners)
totalRecords(df_partners, 'Partners')

Total Partners: 2046430


### Treatment of the Companies Dataframe

In [7]:
df_companies.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,4519,DANIELA DA SILVA CRUZ,2135,50,"0,00",5,None
1,8638,JOAO DOS SANTOS FAGUNDES,2135,50,"0,00",5,None
2,11748,PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO...,2062,49,"0,00",1,None
3,12027,L G SORVETERIA LTDA,2062,49,"0,00",5,None
4,13289,ANDREIA CRISTINA DELSIN EIRELI,2305,65,"100000,00",1,None


In [8]:
df_companies.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [9]:
df_companies.select(
    [func.count(func.when(func.isnull(c), 1)).alias(c) for c in df_companies.columns]).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,0,0,0,0,0,5985,4579678


In [10]:
df_companies = replaceValues(df_companies, 'capital_social_da_empresa', ',', '.')
df_companies = convertType(df_companies, 'capital_social_da_empresa', DoubleType())

In [11]:
df_companies.select(func.concat_ws(' - ', 'razao_social_nome_empresarial', 'cnpj_basico').alias('NOME - CNPJ')).show(5, truncate=False)

+-----------------------------------------------------------+
|NOME - CNPJ                                                |
+-----------------------------------------------------------+
|DANIELA DA SILVA CRUZ - 4519                               |
|JOAO DOS SANTOS FAGUNDES - 8638                            |
|PANIFICADORA E CONFEITARIA CONFIANCA RIO PRETO LTDA - 11748|
|L G SORVETERIA LTDA - 12027                                |
|ANDREIA CRISTINA DELSIN EIRELI - 13289                     |
+-----------------------------------------------------------+
only showing top 5 rows



In [12]:
df_companies.select(
                    func.concat_ws(', ', \
                                   func.substring_index(col("razao_social_nome_empresarial"), ' ', -1),
                                   func.substring_index(col("razao_social_nome_empresarial"), ' ', 1)) \
                                   .alias('Sobrenome, Nome'), 
                                   'cnpj_basico').show(5)

+------------------+-----------+
|   Sobrenome, Nome|cnpj_basico|
+------------------+-----------+
|     CRUZ, DANIELA|       4519|
|    FAGUNDES, JOAO|       8638|
|LTDA, PANIFICADORA|      11748|
|           LTDA, L|      12027|
|   EIRELI, ANDREIA|      13289|
+------------------+-----------+
only showing top 5 rows



### Treatment of the Establishments Dataframe

In [13]:
df_establishments.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,4519,1,48,1,GIRAFFAS,8,19950331,1,None,NaN,...,6219,None,None,None,None,NaN,None,None,None,NaN
1,8638,1,79,1,AGROPECUARIA FAGUNDES,8,20150209,73,None,NaN,...,7255,None,None,None,None,NaN,None,None,None,NaN
2,11748,1,90,1,None,4,20181219,63,None,NaN,...,7097,None,None,None,None,NaN,None,None,None,NaN
3,12027,1,2,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,13289,1,83,1,JS MATERIAIS DE CONSTRUCAO,2,20040123,0,None,NaN,...,6915,19,35811286,None,None,NaN,None,CONTATO@LEONECONTABIL.COM.BR,None,NaN


In [14]:
df_establishments.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: str

In [15]:
df_establishments.select(
    [func.count(func.when(func.isnull(c), 1)).alias(c) for c in df_establishments.columns]).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,0,0,0,0,2014706,0,0,0,4833519,4823786,...,0,1251627,1251623,4424413,4425166,4132454,4131990,2304672,4836208,4836208


In [16]:
df_establishments = convertType(df_establishments, 'data_situacao_cadastral', StringType())
df_establishments = formatStringToDate(df_establishments, 'data_situacao_cadastral')
df_establishments.select('data_situacao_cadastral').limit(5).toPandas()

,data_situacao_cadastral
0,31/03/1995
1,09/02/2015
2,19/12/2018
3,31/12/2008
4,23/01/2004


In [17]:
df_establishments = convertType(df_establishments, 'data_de_inicio_atividade', StringType())
df_establishments = formatStringToDate(df_establishments, 'data_de_inicio_atividade')
df_establishments.select('data_de_inicio_atividade').limit(5).toPandas()

,data_de_inicio_atividade
0,16/05/1994
1,24/05/1994
2,31/05/1994
3,08/06/1994
4,01/06/1994


In [18]:
df_establishments = convertType(df_establishments, 'data_da_situacao_especial', StringType())
df_establishments = formatStringToDate(df_establishments, 'data_da_situacao_especial')
df_establishments.filter("data_da_situacao_especial is not Null").toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,3302548,1,84,1,LAURA MATERIAIS DE LIMPEZA E HIDRAULICOS,8,15/09/2020,1,None,NaN,...,7107,11,37443180,None,None,NaN,None,contabil-camargo@uol.com.br,ESPOLIO EV 407,02/07/2020
1,2191115,1,36,1,None,2,25/02/2001,0,None,NaN,...,7535,041,3226355,None,None,41.0,2331695,None,RECUPERACAO JUDICIAL,22/12/2020
2,496012,1,59,1,None,4,23/03/2021,63,None,NaN,...,8511,051,3183075,None,None,NaN,None,None,ESPOLIO EV 407,26/04/2021
3,5043843,1,99,1,None,2,03/11/2005,0,None,NaN,...,2921,22,27643069,22,27646906,NaN,None,CONTDP@VELOXMAIL.COM.BR,ESPOLIO EV 407,23/02/2021
4,13157,1,51,1,None,2,03/11/2005,0,None,NaN,...,6619,None,None,None,None,NaN,None,None,FALIDO,03/11/2016
5,6514,1,54,1,FEIRINHA POPULAR,2,03/11/2005,0,None,NaN,...,6575,16,31461384,None,None,NaN,None,None,ESPOLIO EV 407,21/07/2020
6,94760634,1,96,1,CIA DAS FECHADURAS,8,19/02/2021,1,None,NaN,...,8877,51,35903362,None,None,51.0,35903362,regismetz@sinos.net,ESPOLIO EV 407,04/01/2021
7,3309171,1,95,1,CYCLO BYKE,8,16/03/2018,1,None,NaN,...,6389,None,None,None,None,NaN,None,None,ESPOLIO EV 407,11/12/2017
8,5882398,1,50,1,CAFE & BOMBONIERE ALMEIDA,2,07/10/2005,0,None,NaN,...,7107,11,64010155,None,None,NaN,None,solucao@solucaoc.com.br,ESPOLIO EV 407,04/05/2021
9,74551532,1,53,1,None,2,27/11/1999,0,None,NaN,...,6477,11,64088588,None,None,NaN,None,None,LIQUIDACAO EXTRA-JUDICIAL,26/04/2021


In [19]:
df_establishments.select('data_da_situacao_especial', year(to_date(col('data_da_situacao_especial'), 'dd/MM/yyyy')).alias('year')).where("data_da_situacao_especial is not Null").toPandas()

,data_da_situacao_especial,year
0,02/07/2020,2020
1,22/12/2020,2020
2,26/04/2021,2021
3,23/02/2021,2021
4,03/11/2016,2016
5,21/07/2020,2020
6,04/01/2021,2021
7,11/12/2017,2017
8,04/05/2021,2021
9,26/04/2021,2021


### Treatment of the Partners Dataframe

In [20]:
df_partners.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [21]:
df_partners.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [22]:
df_partners.select(
    [func.count(func.when(func.isnull(c), 1)).alias(c) for c in df_partners.columns]).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,0,0,208,1234,0,0,2038255,0,1995432,0,0


In [23]:
df_partners = convertType(df_partners, 'data_de_entrada_sociedade', StringType())
df_partners = formatStringToDate(df_partners, 'data_de_entrada_sociedade')
df_partners.select('data_de_entrada_sociedade').limit(5).toPandas()

,data_de_entrada_sociedade
0,25/07/1994
1,25/07/1994
2,16/05/1994
3,16/05/1994
4,09/06/1994


In [24]:
df_partners.fillna(0).limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,25/07/1994,0,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,25/07/1994,0,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,16/05/1994,0,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,16/05/1994,0,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,09/06/1994,0,***000000**,None,0,8


In [25]:
df_partners.fillna('-').limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,25/07/1994,NaN,***000000**,-,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,25/07/1994,NaN,***000000**,-,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,16/05/1994,NaN,***000000**,-,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,16/05/1994,NaN,***000000**,-,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,09/06/1994,NaN,***000000**,-,0,8


### Spark Dataframe Basic Operations

In [26]:
df_partners \
    .select(
        'nome_do_socio_ou_razao_social', 
        'qualificacao_do_socio') \
    .orderBy('qualificacao_do_socio', ascending=False) \
    .limit(5) \
    .toPandas()

,nome_do_socio_ou_razao_social,qualificacao_do_socio
0,LINKED2 GMBH,79
1,"BIZISA HOLDING, S.A.",79
2,GLOBAL TELEMED SYSTEMS AG,79
3,"NEWMARK PARTNERS, L.P.",79
4,MATI RIVER LTD.,79


In [27]:
df_partners \
    .select(
        'nome_do_socio_ou_razao_social', 
        'qualificacao_do_socio', 
        'data_de_entrada_sociedade') \
    .orderBy(['qualificacao_do_socio', to_date('data_de_entrada_sociedade', 'dd/MM/yyyy')], ascending=[False, True]).limit(5).toPandas()

,nome_do_socio_ou_razao_social,qualificacao_do_socio,data_de_entrada_sociedade
0,EDITORIAL MEDICA A.W.W.E. S.A.,79,28/02/2003
1,SANDEN INTERNATIONAL (USA) INC,79,30/06/2006
2,CHUTNEY HOLDINGS LTD,79,28/11/2006
3,UNI-ALIMENTOS S/A,79,14/08/2009
4,LA FINESTRA SUL CIELO ESPA�A SL,79,29/04/2011


In [28]:
df_partners.where(df_partners.nome_do_socio_ou_razao_social.like('%RESTAURANTE%')).limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,7649525,1,GIRAMAIS RESTAURANTES E PARTICIPACOES LTDA.,17280236000121,22,30/08/2013,NaN,***455181**,EDUARDO FREIRE GUERRA,5,0
1,35690629,1,IFOOD.COM AGENCIA DE RESTAURANTES ONLINE S.A.,14380200000121,22,08/01/2020,NaN,***140166**,DIEGO CARNEIRO BARRETO,5,0
2,11802182,1,R. R. BAR E RESTAURANTE LTDA,11802182000130,63,21/02/2011,NaN,***000000**,None,0,0
3,35621563,1,ALPHA YK RESTAURANTE LTDA,23320323000168,31,01/09/2019,NaN,***000000**,None,0,0
4,4836830,1,SAMPAIO 361 RESTAURANTE LTDA,04836830000104,63,02/12/2014,NaN,***000000**,None,0,0


In [29]:
df_partners \
    .select(
        'nome_do_socio_ou_razao_social', 
        'data_de_entrada_sociedade', 
        func.month(to_date(col('data_de_entrada_sociedade'), 'dd/MM/yyyy')).alias('month')) \
    .filter(
        (col('month') >= 6) &
        (col('month') <= 8)
).limit(5).toPandas()

,nome_do_socio_ou_razao_social,data_de_entrada_sociedade,month
0,LILIANA PATRICIA GUASTAVINO,25/07/1994,7
1,CRISTINA HUNDERTMARK,25/07/1994,7
2,HANNE MAHFOUD FADEL,09/06/1994,6
3,CLOD ASSAD FADEL,09/06/1994,6
4,SEBASTIAO JADIR TEIXEIRA NUNES,13/08/2009,8


In [30]:
df_partners \
    .select(
        'nome_do_socio_ou_razao_social', 
        'data_de_entrada_sociedade',
        func.month(to_date(col('data_de_entrada_sociedade'), 'dd/MM/yyyy')).alias('month')) \
    .filter(
        ("month >= 6 and month <= 8")
).limit(5).toPandas()

,nome_do_socio_ou_razao_social,data_de_entrada_sociedade,month
0,LILIANA PATRICIA GUASTAVINO,25/07/1994,7
1,CRISTINA HUNDERTMARK,25/07/1994,7
2,HANNE MAHFOUD FADEL,09/06/1994,6
3,CLOD ASSAD FADEL,09/06/1994,6
4,SEBASTIAO JADIR TEIXEIRA NUNES,13/08/2009,8


In [31]:
df_partners \
    .select(
        year(to_date('data_de_entrada_sociedade', 'dd/MM/yyyy')).alias('year'), 
        'faixa_etaria') \
    .groupBy('year') \
    .agg(
        func.count('year').alias('total_ano'),
        func.round(func.avg("faixa_etaria"), 0).alias('faixa_etaria_media')
    ) \
    .orderBy('year', ascending=False) \
    .limit(5) \
    .toPandas()

,year,total_ano,faixa_etaria_media
0,2021,56316,4.0
1,2020,125927,4.0
2,2019,118248,5.0
3,2018,99935,5.0
4,2017,90221,5.0


In [32]:
df_companies.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [33]:
df_partners.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [34]:
df_companies.join(df_partners, 'cnpj_basico', how='inner').printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [35]:
count = df_partners \
        .select(year(to_date('data_de_entrada_sociedade', 'dd/MM/yyyy')).alias('year')) \
        .where('year >= 2010') \
        .groupBy('year') \
        .count() \
        .orderBy('year', ascending=False)
count.toPandas()

,year,count
0,2021,56316
1,2020,125927
2,2019,118248
3,2018,99935
4,2017,90221
5,2016,81587
6,2015,80906
7,2014,80590
8,2013,83919
9,2012,80101


In [36]:
count.union(
    count.select(
        func.lit('Total'),
        func.sum('count'))
).toPandas()

,year,count
0,2021,56316
1,2020,125927
2,2019,118248
3,2018,99935
4,2017,90221
5,2016,81587
6,2015,80906
7,2014,80590
8,2013,83919
9,2012,80101


### SparkSQL

In [37]:
df_partners.createOrReplaceTempView('partnersView')

In [38]:
spark.sql('SELECT * FROM partnersView LIMIT 5').toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,25/07/1994,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,25/07/1994,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,16/05/1994,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,16/05/1994,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,09/06/1994,NaN,***000000**,None,0,8
